In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/data/Data_PedRegaTrauma_coded_for_analysis_250417.xlsx'

In [ ]:
data_df = pd.read_excel(data_path, sheet_name='All centres cleaned')
data_df['age'] = (pd.to_datetime(data_df['Date of accident']) - pd.to_datetime(data_df['Date of Birth']))/pd.Timedelta(days=365.25)

In [ ]:
# mechanism encoding
# Trafic accident	1
# Fall (from hights)	2
# Burning/scalding	3
# battled child	4
# Kollision mit Beteiligung anderer Menschen	5
# Skisturz/ Hochrasantstrauma/ Kollision mit Gegenstände	6
# Stolpersturz/ Sturz aus Körperhöhe	7
# other traumatic impact	8

# should be grouped together:
# collisions: 1, 5, 6
# falls: 2, 7
# burns: 3
# other: 4, 8

mechanism_encoding_groups_map = {
    'collisions': [1, 5, 6],
    'falls': [2, 7],
    'burns': [3],
    'other': [4, 8]
}

In [ ]:
data_df['mechanism_group'] = data_df['Mechanism of injury'].map(lambda x: next((group for group, codes in mechanism_encoding_groups_map.items() if x in codes), pd.NA))

In [ ]:
data_df['Mechanism of injury'].value_counts()

In [ ]:

# fig, ax = plt.subplots(figsize=(10, 6))
fig = plt.figure(figsize=(10, 6))
g = sns.displot(data_df, x='age', hue='mechanism_group', kind='kde', fill=True, palette=['#E31837', '#0072CE', '#8B4513', '#D2691E'], alpha=0.1, height=6, aspect=1.5)
g.ax.set_xlim(0, 18)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()
groups = data_df['mechanism_group'].unique()
colors = ['#2E8B57', '#FF6347', '#4682B4', '#DAA520']  # Sea green, tomato, steel blue, goldenrod

for i, group in enumerate(groups):
    if i < len(axes):
        subset = data_df[data_df['mechanism_group'] == group]['age']
        # axes[i].hist(subset, bins=15, alpha=0.7, edgecolor='black', color=colors[i])
        axes[i] = sns.histplot(subset, bins=15, kde=False, color=colors[i], ax=axes[i], stat='count', alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{group.capitalize()} (n={len(subset)})')
        axes[i].set_xlabel('Age (years)')
        axes[i].set_ylabel('Count')
        axes[i].set_xlim(0, 18)  # Same xlim for all subplots
        axes[i].set_xticks(range(0, 19, 2))  # Integer x-ticks
        axes[i].yaxis.set_major_locator(plt.MaxNLocator(integer=True))  # Integer y-ticks
        # remove top and right spines
        axes[i].spines['top'].set_visible(False)
        axes[i].spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# fig.savefig('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/figures/mecchanism_age_distribution.png', dpi=600, bbox_inches='tight')